# 7.2 PDE-Constrained Shape Optimization in NGSolve

We want to solve the PDE-constrained shape optimization problem
\begin{equation}
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega |u-u_d|^2 \; dx
\end{equation}
 subject to that $(\Omega,u)$ satisfy
 \begin{equation}
           \int_\Omega \nabla u \cdot \nabla v \; dx = \int_\Omega f v \; dx \; \quad \text{ for all } v \in H_0^1(\Omega),
\end{equation}
where $\Omega \subset \mathbb R^2$ for given $u_d, f \in C^1(\mathbb R^2)$.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

In [ ]:
geo = SplineGeometry()
geo.AddRectangle( (0.2, 0.2), (0.8, 0.8), bcs = ('b','r','t','l'))
mesh = Mesh(geo.GenerateMesh(maxh = 0.1))

In [ ]:
#given data of our problem (chosen such that \Omega^* = [0,1]^2 is the optimal shape)
f = CoefficientFunction(2*y*(1-y)+2*x*(1-x))
ud = x*(1-x)*y*(1-y)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

### State equation

In [ ]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene = Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

fstate = LinearForm(fes)
fstate += f*v*dx

In [ ]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [ ]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
scene.Redraw()

### Adjoint equation

We set up the adjoint equation
\begin{align*}
    \mbox{Find } p \in H_0^1(\Omega): \int_\Omega \nabla w \cdot \nabla p \, \mbox dx = - \partial_u J(u)(w) \quad \text{ for all } w \in H_0^1(\Omega)
\end{align*}
where $u$ is the solution to the state equation. For $J(u) = \int_\Omega |u-u_d|^2 \mbox dx$, we get
\begin{align*}
    \partial_u J(u)(w) = -2 \int_\Omega (u-u_d)w \,\mbox dx.
\end{align*}
However, we can also use the Diff(...) command:

In [ ]:
def Cost(u): 
    return (u-ud)**2*dx

p, w = fes.TnT()
gfp = GridFunction(fes)
scene = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
fadjoint += -1*Cost(gfu).Diff(gfu,w)

In [ ]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [ ]:
fadjoint.Assemble()
SolveAdjointEquation()
scene.Redraw()

Note that (for linear problems) the operator on the left hand side of the adjoint equation is just the transpose of the state operator.

### Shape derivative

The shape derivative of the shape function 
$$
\mathcal{J}(\Omega):= \int_\Omega |u-u_d|^2\;dx
$$
at $\Omega$ in direction $X$ is given by the formula
\begin{align*}
    d\mathcal J(\Omega; X) =&\int_{\Omega}  \mbox{div}(X) |u-u_d|^2  - 2(u-u_d)\nabla u_d \cdot X \, dx \\
		& + \int_{\Omega} (\mbox{div}(X) I - D X - D X^\top )\nabla u \cdot \nabla p \, dx \\
		&- \int_\Omega (\nabla f\cdot X + f \mbox{div}(X)) p\, dx.
\end{align*}
We now assemble this shape derivaitve in NGSolve as follows:

In [ ]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

dJOmega = LinearForm(VEC)
dJOmega += SymbolicLFI(div(X)*( (gfu-ud)**2 - f*gfp + InnerProduct(grad(gfu),grad(gfp))))
dJOmega += SymbolicLFI(-2*(gfu-ud)*InnerProduct(grad_ud,X) - InnerProduct(grad_f,X)*gfp)
dJOmega += SymbolicLFI(-InnerProduct(X.Deriv()*grad(gfu),grad(gfp))-InnerProduct(grad(gfu),X.Deriv()*grad(gfp)))

### Descent Direction

<ul>
 <li> Next, we want to find a vector field $X$ which yields a decrease of the objective function, i.e. we want to find a vector field $X$ such that $$d \mathcal J(\Omega;X)<0.$$ </li>
<li> This can be achieved by solving an auxiliary boundary value problem of the type 
\begin{align*}
    \mbox{Find } X \in H: \qquad b(X, \Phi) = d\mathcal J(\Omega; \Phi) \; \text{ for all } \Phi \in H
\end{align*}
for a suitable Hilbert space $H$ (e.g. $H=H^1(\Omega)^2$). Here, $b(\cdot, \cdot): H \times H \rightarrow \mathbb R$ is a positive definite bilinear form which we are free to choose. Then, $-X$ is a descent direction since $$ d\mathcal J(\Omega; -X) = -d\mathcal J(\Omega; X) = - b(X, X) < 0.$$ </li>
</ul>

In [ ]:
b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

In [ ]:
def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [ ]:
b.Assemble()
dJOmega.Assemble()
SolveDeformationEquation()
Draw(-gfX, mesh, "gfX")

Now let us update the geometry in the direction of $-X$. Here, we need to choose the step size accordingly (a descent is only guaranteed for this step size being sufficiently small)

In [ ]:
print('Cost at initial design', Integrate (Cost(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

Let us now update the design and compute the new value of the cost function:

In [ ]:
scene = Draw(gfset)
mesh.SetDeformation(gfset)
scene.Redraw()

a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (Cost(gfu), mesh))

In [ ]:
#reset the design
gfset.Set((0,0))
mesh.SetDeformation(gfset)

#check if initial value of cost function 0.000486578350214552 is recovered
a.Assemble()
fstate.Assemble()
SolveStateEquation()
print('Cost at new design', Integrate (Cost(gfu), mesh))

Finally, this can be applied in an iterative algorithm with a line search for choosing the step size:

In [ ]:
scene = Draw(gfset)

In [ ]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
a.Assemble()
fstate.Assemble()
SolveStateEquation()

LineSearch = True

iter_max = 600
Jold = Integrate(Cost(gfu), mesh)
converged = False

# input("Press enter to start optimization")
for k in range(iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    print('cost at iteration', k, ': ', Jold)
    
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmega.Assemble()
    SolveDeformationEquation()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(Cost(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            print("scale = ", scale)
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            fstate.Assemble()
            SolveStateEquation()
            Jnew = Integrate(Cost(gfu), mesh)
    
    if converged==True:
        print("No more descent can be found")
        break
    Jold = Jnew

    Redraw(blocking=True)